In [62]:
import tensorflow as tf
import tensorflow.keras.models as models
import pickle
import numpy as np
import pandas as pd

In [63]:
### Load the trained model , scaler picked file and one hot encoder picked file
model = models.load_model('model.h5')

### Load the encoder and scaler
with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

#That's why we stored ohe and label encoder after fitting so that we can use them in any file in future the changes need not be done again

with open('label_encoder_gender.pkl', 'rb') as f:
    label_encoder_gender= pickle.load(f) 

with open('onehot_encoder_geo.pkl', 'rb') as f:
    label_encoder__geo = pickle.load(f)


In [64]:
#Example input data for prediction
input_data = {
    'CreditScore': 750,
    'Geography': 'France',
    'Gender' : 'Male',
    'Age': 35,
    'Tenure': 5,
    'Balance': 100000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [65]:
# One-hot encode 'Geography' feature
geo_encoded = label_encoder__geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=label_encoder__geo.get_feature_names_out(['Geography']))
geo_encoded_df

c:\Users\onlyy\OneDrive\Documents\GenAI_ANN_Project1\venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [66]:
### Combine one-hot encoded columns with input data
input_data = pd.concat([pd.DataFrame([input_data]), geo_encoded_df], axis=1)
input_data.drop('Geography', axis=1, inplace=True)
input_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,750,Male,35,5,100000,2,1,1,50000,1.0,0.0,0.0


In [67]:
### Encode my categorical variables
input_data['Gender'] = label_encoder_gender.transform(input_data[['Gender']])
input_data

c:\Users\onlyy\OneDrive\Documents\GenAI_ANN_Project1\venv\Lib\site-packages\sklearn\preprocessing\_label.py:139: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,750,1,35,5,100000,2,1,1,50000,1.0,0.0,0.0


In [55]:
### Concatination of ohe 
input_data = pd.concat ([input_data , geo_encoded_df] , axis = 1) 
input_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain,Geography_France,Geography_Germany,Geography_Spain
0,750.0,1.0,35.0,5.0,100000.0,2.0,1.0,1.0,50000.0,1.0,0.0,0.0,1.0,0.0,0.0
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,1.0,0.0,0.0


SCALING:

Scaling in the context of machine learning refers to the process of adjusting the range of feature values so that they have a consistent scale. This is crucial for various reasons:
1. Normalization of Input Values.
2. Preventing Large Values.
3. Improved Training Stability

In [68]:
### Scaling the input data
input_scale = scaler.transform(input_data)
input_scale

array([[ 1.0206745 ,  0.91324755, -0.37056859, -0.00134472,  0.38262839,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [69]:
### Predict whether person will leave the bank or not
prediction=model.predict(input_scale)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step


array([[0.01538832]], dtype=float32)

In [73]:
prediction_probability = prediction[0][0]
if prediction_probability > 0.5:
    print("The customer is predicted to leave")
else:
    print("The customer is not likely to leave")


The customer is not likely to leave
